# Calling WGDs

If you run MEDICC2 in the standard (and recommended way) from the command line, the WGD status will be outputed both in the `_summary.tsv` file as well as in the `_copynumber_events_df.tsv` file (these are calcualted using approach 1, see below).

In this notebook, we shows how to call WGDs inside python using MEDICC2.
There are two approaches to do this. The standard way (Approach 1) runs MEDICC2's main routine which reconstruct a phylogentic tree of the samples. It then reconstruct the ancestral states (i.e. internal nodes) and infers events for every branch of the tree. If MEDICC2 detects a WGD in a branch, all nodes that lie downstream of that branch are considered to have a WGD.

In the second approach we calculate the optimal copy-number events directly from the diploid state for every sample individually. In the case that the phylogenetic tree and the ancestral states are correct, this leads to exactly the same events and therefore the same WGD calls as in approach 1.
In some cases however, MEDICC2 is not able to accurately infer the tree structure and/or the ancestral states. This is mainly the case for single-cell experiments with 100s-1000s of samples and a high noise-to-signal ratio. It is possible that MEDICC2 misses a WGD in the corresponding branch.

If the WGD detection differs between the two approaches, there is likely a mistake with the phylogeny creation, i.e. the reconstructed tree is not accurate. In this, we would recommened to rather trust the WGD calls that were created per sample and not from the tree.

# Imports and definitions

In [1]:
#%% import and load data
import os
from pathlib import Path
import sys
import pandas as pd

sys.path.append('..')
import medicc
import fstlib

In [2]:
fst = medicc.io.read_fst()
fst_nowgd = medicc.io.read_fst(no_wgd=True)
symbol_table = fst.input_symbols()

# Run WGD detection for all samples in Gundem et al. 2015

In [3]:
data_folder = "../examples/gundem_et_al_2015"
                   
patients = [f.split('_')[0] for f in os.listdir(data_folder) if 'input_df.tsv' in f]
patients.sort()

In [4]:
results = dict()

for patient in patients:
    # Load data
    input_df = medicc.io.read_and_parse_input_data(
        os.path.join(data_folder, "{}_input_df.tsv".format(patient)))

    # Create DataFrame
    wgd_status = pd.DataFrame(index=input_df.index.get_level_values('sample_id').unique())
    wgd_status['in_tree'] = False
    wgd_status['individually'] = False

    # Approach 1: Create phylogenetic tree and infer WGD status from tree
    # run MEDICC2's main routine 
    sample_labels, pairwise_distances, nj_tree, final_tree, output_df, events_df = medicc.main(
        input_df,
        fst)

    # Find WGD nodes from the phylogenetic tree
    wgd_nodes = events_df.loc[events_df['type'] == 'wgd'].index.get_level_values(0).unique()
    for wgd_node in wgd_nodes:
        wgd_status.loc[[x.name for x in list(final_tree.find_clades(wgd_node))[0].get_terminals() if x.name is not None], 'in_tree'] = True
    
    # Approach 2: Calculate WGD status for each sample individually
    # Create FSAs for diploid and samples
    diploid_fsa = medicc.tools.create_diploid_fsa(fst)
    fsa_dict = medicc.create_standard_fsa_dict_from_data(input_df[['cn_a', 'cn_b']], symbol_table, 'X')

    # Calc distance (ie number of events from diploid) without WGD
    dist_no_wgd = pd.Series({aliquot_id: float(fstlib.score(fst_nowgd, diploid_fsa, fsa))
                                for aliquot_id, fsa in fsa_dict.items()}, name='dist_no_wgd_a')

    # Calc distance with WGD
    dist_wgd = pd.Series({aliquot_id: float(fstlib.score(fst, diploid_fsa, fsa))
                            for aliquot_id, fsa in fsa_dict.items()}, name='dist_wgd_a')
    
    # WGD status is True if distance with WGD is smaller than without WGD
    wgd_status.loc[dist_wgd.loc[dist_no_wgd != dist_wgd].index, 'individually'] = True
    results[patient] = wgd_status

2022-10-17 15:20:21,501 - WARNING medicc.io:	Integer CN > maxcn 8, capping.
2022-10-17 15:20:21,503 - WARNING medicc.io:	Integer CN > maxcn 8, capping.
2022-10-17 15:21:11,832 - WARNING medicc.io:	Integer CN > maxcn 8, capping.
2022-10-17 15:21:11,833 - WARNING medicc.io:	Integer CN > maxcn 8, capping.


## Results

In [5]:
for patient in patients:
    print(patient)
    if results[patient].any().any():
        print('WGD detected')
        display(results[patient])
    else:
        print('no WGD detected')
    print('')

PTX004
no WGD detected

PTX005
no WGD detected

PTX006
no WGD detected

PTX007
WGD detected


,in_tree,individually
sample_id,,
LAdrenalMet_A21D-0096_CRUK_PC_0096_M3,False,False
LClavicleLNMet_A21I-0096_CRUK_PC_0096_M8,False,False
LIliacCrestSoftMet_A21J-0096_CRUK_PC_0096_M9,False,False
LRib5MassMet_A21A-0096_CRUK_PC_0096_M1,False,False
MultLiverMet13_A21H-0096_CRUK_PC_0096_M7,False,False
RRibNodularMet_A21F-0096_CRUK_PC_0096_M5,True,True
SingleLiverMet2_A21G-0096_CRUK_PC_0096_M6,False,False
SingleLiverMet4_A21E-0096_CRUK_PC_0096_M4,False,False
SingleLiverMet8_A21C-0096_CRUK_PC_0096_M2,False,False



PTX008
no WGD detected

PTX009
no WGD detected

PTX010
WGD detected


,in_tree,individually
sample_id,,
ProstateCA_A29C-0017_CRUK_PC_0017_T1,True,True
RSuperficialInguinalLNMetA1_A29A-0017_CRUK_PC_0017_M1,True,True
diploid,False,False



PTX011
WGD detected


,in_tree,individually
sample_id,,
LAdrenalMet_A31E-0018_CRUK_PC_0018_M3,True,True
Prostate1-1-2CA_A31C-0018_CRUK_PC_0018_T1,False,False
RIngLNMet_A31D-0018_CRUK_PC_0018_M2,True,True
RRib7Met_A31F-0018_CRUK_PC_0018_M4,True,True
RSubduralMet_A31A-0018_CRUK_PC_0018_M1,True,True
diploid,False,False



PTX012
WGD detected


,in_tree,individually
sample_id,,
LCervicalLNMet1-2_A32D-0019_CRUK_PC_0019_M2,True,True
LSubclavicularLNMet1-5_A32E-0019_CRUK_PC_0019_M3,True,True
Prostate10-1-3CA_A32C-0019_CRUK_PC_0019_T1,True,True
RHumerusMet1-12_A32F-0019_CRUK_PC_0019_M4,True,True
RRib8Met1_A32A-0019_CRUK_PC_0019_M1,True,True
diploid,False,False



PTX013
no WGD detected



NOTE: Here we find 4 patients with WGDs. Both approaches lead to the same results for all patients which is expected for small trees and high quality copy-number inputs.